# Análisis ponderado de acciones de S&P a mediano plazo
## Este programa utiliza Yahoo Finance para analizar y ponderar todas las acciones de S&P 500 en los últimos 5 años para luego visualizar las mejores 10 inversiones a mediano plazo

### Librerías necesarias

In [4]:
!pip install yfinance
!pip install pandas_ta
!pip install plotly

import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from datetime import datetime

### Descarga de datos con Yahoo Finance

In [112]:
def get_data(ticker, start_date='2020-01-01'):
    # Obtener la fecha actual
    end_date = datetime.today().strftime('%Y-%m-%d')

    # Obtener datos históricos
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    return data

### Cálculo de indicadores técnicos y muestra

In [113]:
def calculate_indicators(data, ticker):
    # Calcular MACD (diferencia entre EMA 12 y EMA 26)
    data['MACD'], data['MACD_signal'], _ = ta.macd(data[('Close', ticker)])

    # Calcular RSI (Índice de Fuerza Relativa)
    data['RSI'] = ta.rsi(data[('Close', ticker)])

    # Calcular ADX (Average Directional Index, mide la fuerza de la tendencia)
    adx_result = ta.adx(data[('High', ticker)], data[('Low', ticker)], data[('Close', ticker)])
    data[('ADX', ticker)] = adx_result['ADX_14']  # Asignar ADX
    data[('ADX_pos', ticker)] = adx_result['DMP_14']  # Asignar ADX Positive (DMP_14)
    data[('ADX_neg', ticker)] = adx_result['DMN_14']  # Asignar ADX Negative (DMN_14)

    # Calcular Bandas de Bollinger (Medimos si el precio está en zona de sobrecompra/sobreventa)
    bbands_result = ta.bbands(data[('Close', ticker)])
    data[('Upper_Band', ticker)] = bbands_result['BBU_5_2.0']  # Asignar Upper Band
    data[('Lower_Band', ticker)] = bbands_result['BBL_5_2.0']  # Asignar Lower Band

    return data

# Muestra cómo quedan los indicadores para la acción AAPL:
ticker = 'AAPL'
data = get_data(ticker)
data_with_indicators = calculate_indicators(data, ticker)
data_with_indicators[['MACD', 'RSI', 'ADX', 'Upper_Band', 'Lower_Band']].tail()

Price,MACD,RSI,ADX,Upper_Band,Lower_Band
Ticker,,,AAPL,AAPL,AAPL
Date,,,,,
2025-03-03,MACD_12_26_9,48.447005,20.376333,247.846277,233.981719
2025-03-04,MACD_12_26_9,45.885176,19.598156,242.953143,234.430854
2025-03-05,MACD_12_26_9,45.649990,19.759878,242.182507,233.353491
2025-03-06,MACD_12_26_9,45.112604,19.612410,242.216187,232.531811
2025-03-07,MACD_12_26_9,50.801949,18.720497,239.746463,233.893539


### Obtención de métricas fundamentales

In [122]:
def get_fundamentals(ticker):
    stock = yf.Ticker(ticker)
    # Obtener el reporte financiero
    try:
        fundamentals = stock.info
        pe_ratio = fundamentals.get('trailingPE', None)  # P/E ratio
        pb_ratio = fundamentals.get('priceToBook', None)  # P/B ratio
        roe = fundamentals.get('returnOnEquity', None)  # ROE

        return pe_ratio, pb_ratio, roe
    except Exception as e:
        print(f"Error obteniendo datos fundamentales para {ticker}: {e}")
        return None, None, None

### Definir los pesos para cada métrica

In [132]:
def calculate_weighted_score(data, ticker):
    # Asignar pesos a cada indicador
    weight_macd = 0.3
    weight_rsi = 0.2
    weight_adx = 0.2
    weight_bbands = 0.3
    weight_pe = 0.15
    weight_pb = 0.1
    weight_roe = 0.1

    # Normalizar y combinar los indicadores (simplificación)
    macd_score = (data['MACD'] > data['MACD_signal']).mean()
    rsi_score = (data['RSI'] < 30).mean()  # Sobreventa
    adx_score = (data['ADX'] > 25).mean()  # Tendencia fuerte
    bbands_score = ((data['Close'] > data['Upper_Band']).mean())  # Precio en la banda superior

    # Obtener datos fundamentales
    pe_ratio, pb_ratio, roe = get_fundamentals(ticker)

    # Calcular puntajes fundamentales
    pe_score = 1 if pe_ratio and pe_ratio < 15 else 0  # Bajo P/E es bueno
    pb_score = 1 if pb_ratio and pb_ratio < 1 else 0   # Bajo P/B es bueno
    roe_score = 1 if roe and roe > 0.15 else 0  # ROE positivo y alto es bueno

    # Normalizar puntajes
    score = (weight_macd * macd_score +
               weight_rsi * rsi_score +
               weight_adx * adx_score +
               weight_bbands * bbands_score +
               weight_pe * pe_score +
               weight_pb * pb_score +
               weight_roe * roe_score)

    return score

# Ejemplo de puntaje ponderado
score = calculate_weighted_score(data_with_indicators, ticker)
score

,0
Ticker,
AAPL,0.200614


# Análisis final del ticker

In [129]:
def analyze_tickers(tickers):
    results = []

    for ticker in tickers:
        data = get_data(ticker)
        if data is None or data.empty:
            continue  # Si no se pueden obtener datos, se omite este ticker
        data_with_indicators = calculate_indicators(data, ticker)

        # Calcular el puntaje ponderado
        score = calculate_weighted_score(data_with_indicators, ticker)
        results.append((ticker, score))

    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame(results, columns=['Ticker', 'Puntaje'])
    return results_df

### Obtener tickers del S&P 500 desde Wikipedia

In [130]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

# La primera tabla contiene la lista de tickers
sp500_tickers = tables[0]['Symbol'].tolist()

# Análisis de todas las acciones del S&P 500

In [134]:
sp500_analysis = analyze_tickers(sp500_tickers)
sp500_analysis_sorted = sp500_analysis.sort_values(by='Puntaje', ascending=False)
sp500_analysis_sorted.head(10)

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AOS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ABT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ABBV']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ACN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ADBE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AMD']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AES']: YFRateLimitError('Too 

KeyboardInterrupt: 